In [307]:
# TODO
# Add upper slessor lakes (get outlines from matt)
# Add PIG and AP lakes (find citations) if not too close to grounding line
# Should Carter and others, 2013 citation be changed to Fricker and Scambos, 2009? no, redefined as a lake complex? read carter paper or listen to voice memo

In [308]:
# This notebook collates Antarctic active subglacial lakes from past outline inventories 
# (Smith and others, 2009; Siegfried & Fricker, 2018) as well as point data of lakes in the latest inventory 
# (Livingstone and others, 2022) as well as individual studies since.
#
# Written 2023-01-17 by W. Sauthoff (sauthoff@mines.edu)

In [309]:
# Install package
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [310]:
# Import packages
import fiona
import geopandas as gpd
import h5py
import math
import os
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon, MultiPolygon
from pyproj import CRS, Geod, Transformer

# Define data and script directories dependent on home environment
if os.getenv('HOME') == '/home/jovyan':
    DATA_DIR = '/home/jovyan/data'
    SCRIPT_DIR = '/home/jovyan/repos_my/script_dir'
elif os.getenv('HOME') == '/Users/Wilson': 
    DATA_DIR = '/Volumes/ExtremeSSD/data'
    SCRIPT_DIR = '/Users/Wilson/Documents/0-code/repos_my/script_dir'

# Define utility functions
def ll2ps(lon, lat):
    """
    Transform coordinates from geodetic coordinates (lon, lat)
    to Antarctic Polar Stereograph coordinates (x, y)
    x, y = ll2ps(lon, lat)
    """
    crs_ll = CRS("EPSG:4326")
    crs_xy = CRS("EPSG:3031")
    ll_to_xy = Transformer.from_crs(crs_ll, crs_xy, always_xy = True)
    x, y = ll_to_xy.transform(lon, lat)
    return x, y

def ps2ll(x, y):
    """
    Transform coordinates from Antarctic Polar Stereograph
    to geodetic (lon, lat) coordinates
    
    lon, lat = ps2ll(x, y)
    """
    crs_ll = CRS("EPSG:4326")
    crs_xy = CRS("EPSG:3031")
    xy_to_ll = Transformer.from_crs(crs_xy, crs_ll, always_xy = True)
    lon, lat = xy_to_ll.transform(x, y)
    return lon, lat

def find_intersections(gdf1, gdf2):
    # Create an empty list to store the results
    intersections = []

    # Iterate over each geometry in gdf1
    for index1, geom1 in gdf1.geometry.items():
        # Compare with each geometry in gdf2
        for index2, geom2 in gdf2.geometry.items():
            if geom1.intersects(geom2):
                # If they intersect, add the indices to the list
                intersections.append((index1, index2))

    return intersections

# Import datasets

In [311]:
# Import active subglacial lake outlines from Smith and others (2009) (S09)
# As released in Smith and others, 2012 dataset (https://doi.org/10.15784/601439)
fiona.drvsupport.supported_drivers['KML'] = 'rw'
S09_outlines_lonlat = gpd.read_file(DATA_DIR + '/boundaries/Smith2009_subglacial_lakes/Antarctic_lakes.kml', driver='KML')

# Ensure GeoDataFrame is in EPSG:4326 for geodesic area calculation
if S09_outlines_lonlat.crs != 'EPSG:4326':
    S09_outlines_lonlat.to_crs('EPSG:4326')

# Create a Geod object for calculating area on the WGS84 ellipsoid
geod = Geod(ellps="WGS84")

# Create empty list to store areas
areas = []

# Iterate through each row in the GeoDataFrame
for index, row in S09_outlines_lonlat.iterrows():
    geom = row.geometry

    # Check if the geometry is a MultiPolygon and handle accordingly
    if isinstance(geom, MultiPolygon):
        # Calculate area for each polygon in the MultiPolygon
        area = sum(abs(geod.geometry_area_perimeter(polygon)[0]) for polygon in geom)
    elif isinstance(geom, Polygon):
        # Calculate the area for a single Polygon
        area = abs(geod.geometry_area_perimeter(geom)[0])
    else:
        # Handle other geometry types (if any) or set to NaN or 0
        area = float('nan')  # or 0, depending on how you want to handle errors

    areas.append(area)

# Convert to crs to epsg:3031
S09_outlines = S09_outlines_lonlat.to_crs(3031)

# Append list of areas to geodataframe
S09_outlines['area (m^2)'] = areas

# Delete original geodataframe
del S09_outlines_lonlat

In [312]:
S09_outlines

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08
...,...,...,...,...
119,Whillans_6,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-451544.869 -488823.261 1.000, -45...",7.458477e+07
120,Whillans_7,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-543163.376 -500759.165 1.000, -54...",7.696570e+07
121,Whillans_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-654478.748 -281124.560 1.000, -65...",1.625714e+08
122,Wilkes_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((2214185.180 -666018.604 1.000, 221...",5.880773e+08


In [313]:
# Import active subglacial lake outlines from Siegfried & Fricker (2018) (SF18)
# Original pub: https://doi.org/10.1017/aog.2017.36 
# Code for loading lake outlines available in code bank associated with Siegfried & Fricker (2021), https://doi.org/10.1029/2020GL091089: 
# https://github.com/mrsiegfried/Siegfried2021-GRL/blob/main/data/outlines/load_lakes.ipynb

# import subglacial lake outlines (Siegfried & Fricker, 2018)
h5f = h5py.File(DATA_DIR + '/boundaries/SiegfriedFricker2018_subglacial_lakes/SiegfriedFricker2018-outlines.h5', 'r')
outline_geometries = [] # store polygons
citations = [] # store citation information

# we're going to calculate geodesic lake area because that is often screwed up
# and occasionally incorrect in the literature
areas = []

# we're going to need to do some coordinate transforms for the geodesic area
# define CRS for Antarcica and make a converter from xy to ll
CRS_LL = "EPSG:4326" # wgs84 in lon,lat
CRS_XY = h5f.attrs.get('proj_crs') # get projection from hdf5 file
XY_TO_LL = Transformer.from_crs(CRS_XY, CRS_LL, always_xy = True) # make coord transformer
geod = CRS(CRS_LL).get_geod() # geod object for calculating geodesic area on defined ellipsoid

# look through each lake and load all of it's info
for lake in h5f.keys():
    outline_x = h5f[lake]['x'][:]
    outline_y = h5f[lake]['y'][:]
    outlines_xy = np.stack((outline_x, outline_y),axis=2).reshape(outline_x.shape[1], 2)

    # A single lake with multiple polygons is NaN broken---need to identify and
    # load as a MultiPolygon. Otherwise it's easy (just load as polygon)
    if np.isnan(outlines_xy)[:,0].sum() == 0:
        geometry = Polygon(outlines_xy)
        lon, lat = XY_TO_LL.transform(outlines_xy[:,0], outlines_xy[:,1])
        this_area = abs(geod.polygon_area_perimeter(lon,lat)[0])
    else:
        this_area = 0
        # break at NaN values and load each as separate polygons
        idx = np.where(np.isnan(outlines_xy[:,0]))[0]

        # grab outline of first lake before getting into the loop
        this_outline = outlines_xy[0:idx[0],:]
        pgons = [Polygon(this_outline)] # put the first polygon in a list
        lon,lat = XY_TO_LL.transform(this_outline[:,0], this_outline[:,1])
        this_area += abs(geod.polygon_area_perimeter(lon,lat)[0])/1e6 # add its area
        for i in np.arange(0,len(idx)):
            if i == len(idx)-1:
                this_outline = outlines_xy[idx[i]+1:,:]
            else:
                this_outline = outlines_xy[idx[i]+1:idx[i+1]]

            pgons.append(Polygon(this_outline))
            lon,lat = XY_TO_LL.transform(this_outline[:,0], this_outline[:,1])
            this_area += abs(geod.polygon_area_perimeter(lon,lat)[0])/1e6
        geometry = MultiPolygon(pgons)

    # append all the results in the right place
    outline_geometries.append(geometry)
    citations.append(h5f[lake].attrs.get('citation')[0].decode('UTF-8'))
    areas.append(this_area)

# make a pandas dataframe with all the necessary info
df = pd.DataFrame(zip(h5f.keys(), outline_geometries, areas, citations),
                  columns=['name', 'geometry', 'area (m^2)', 'cite'])
# convert to geopands geodataframe
SF18_outlines = gpd.GeoDataFrame(df, crs=CRS_XY, geometry=outline_geometries)
# close HDF5 file
h5f.close()

In [314]:
SF18_outlines

,name,geometry,area (m^2),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [315]:
# Read subglacial lake point data from Livingstone and others (2022) (L22), https://doi.org/10.1038/s43017-021-00246-9
url = 'https://static-content.springer.com/esm/art%3A10.1038%2Fs43017-021-00246-9/MediaObjects/43017_2021_246_MOESM1_ESM.xlsx'
use_cols = ['Name', 'Lat.  oN', 'Lon. oE', 'Lake Type', 'References', 'Prior Inventory']
import_rows = np.arange(0,676)
L22_points = pd.read_excel(url, sheet_name='Antarctica', usecols=use_cols, skiprows = lambda x: x not in import_rows)

# View just the active lakes of the pandas dataset
L22_activelake_points = L22_points[L22_points['Lake Type'] == 'Active']

# Reset the index, dropping the old one
L22_activelake_points = L22_activelake_points.reset_index(drop=True)

# Display geodataframe
L22_activelake_points

,Name,Lat. oN,Lon. oE,Lake Type,References,Prior Inventory
0,Bindschadler5,-80.610,-123.994,Active,Gray et al. (2005); Smith et al. (2009),Wright & Siegert (2012)
1,Kamb10,-81.446,-120.134,Active,Gray et al. (2005); Smith et al. (2009),Wright & Siegert (2012)
2,L1,-74.000,133.270,Active,Wingham et al. (2006); Surveyed by ICECAP in 2...,Wright & Siegert (2012)
3,U1,-75.940,135.000,Active,Wingham et al. (2006),Wright & Siegert (2012)
4,U2,-76.340,135.770,Active,Wingham et al. (2006),Wright & Siegert (2012)
...,...,...,...,...,...,...
134,THW142,-76.618,-106.185,Active,Smith et al. (2017); Hoffman et al. (2020); Ma...,NaN
135,THW124,-76.462,-106.893,Active,Smith et al. (2017) and Hoffman et al. (2020).,NaN
136,THW70,-75.990,-107.023,Active,Smith et al. (2017) and Hoffman et al. (2020).,NaN
137,WT,-77.300,-114.000,Active,Hoffman et al. (2020).,NaN


In [316]:
# Isolate lakes not previously included in Smith and others (2009; 2017) or Siegfried and Fricker (2018)
L22_activelake_points_new = L22_activelake_points.copy(deep=True)
L22_activelake_points_new.drop(L22_activelake_points_new.loc[L22_activelake_points['References'].str.contains('Smith et al.|Siegfried & Fricker')].index, inplace=True)
L22_activelake_points_new = L22_activelake_points_new.reset_index(drop=True)
# print(len(L22_activelake_points_new))
L22_activelake_points_new

,Name,Lat. oN,Lon. oE,Lake Type,References,Prior Inventory
0,L1,-74.000,133.270,Active,Wingham et al. (2006); Surveyed by ICECAP in 2...,Wright & Siegert (2012)
1,U1,-75.940,135.000,Active,Wingham et al. (2006),Wright & Siegert (2012)
2,U2,-76.340,135.770,Active,Wingham et al. (2006),Wright & Siegert (2012)
3,U3,-76.680,135.930,Active,Wingham et al. (2006),Wright & Siegert (2012)
4,Byrd1 (DL),-81.026,148.302,Active,"Stearns et al., (2008)",Wright & Siegert (2012)
5,Byrd2 (UL),-80.684,146.887,Active,"Stearns et al., (2008)",Wright & Siegert (2012)
6,“Site A”,-67.800,112.540,Active,N. Young (personal comm.),Wright & Siegert (2012)
7,“Site B”,-67.870,113.530,Active,N. Young (personal comm.),Wright & Siegert (2012)
8,“Site C”,-67.840,114.040,Active,N. Young (personal comm.),Wright & Siegert (2012)
9,WT,-77.300,-114.000,Active,Hoffman et al. (2020).,NaN


In [317]:
# Copy Siegfried & Fricker (2018) outline inventory to add entries from more recent publications
lake_locations = SF18_outlines.copy(deep=True)

# Create conditional list of lakes within CryoSat-2 SARIn coverage
CS2SARIn_lakes = ['ConwaySubglacialLake', 'Cook_E1', 'Cook_E2', 'David_1', 'David_s1', 
    'EngelhardtSubglacialLake', 'Foundation_1', 'Institute_E1', 'Institute_W1', 'KT1', 'KT2', 'KT3',
    'Lake10', 'Lake12', 'Lake78', 'Lambert_1', 'LennoxKing_1', 'Mac1', 'Mac2', 'Mac3', 
    'MercerSubglacialLake', 'Ninnis_1', 'Ninnis_2', 'Rec1', 'Rutford_1', 'Slessor_1', 
    'Slessor_23', 'Totten_1', 'Totten_2', 'Thw_70', 'Thw_124', 'Thw_142', 'Thw_170', 
    'UpperSubglacialLakeConway', 'Whillans_6', 'WhillansSubglacialLake', 'Wilkes_1']

# Add new column and populate with whether lake is within CryoSat-2 (CS2) InSAR coverage
lake_locations['CS2_SARIn'] = np.where(lake_locations['name'].isin(CS2SARIn_lakes),'True', 'False')

# Display dataframe preview
lake_locations

,name,geometry,area (m^2),cite,CS2_SARIn
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
...,...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/...",True
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",True


In [318]:
# Change journal abbreviations to ISO4 standard
# First look at citations
lake_locations['cite'].unique()

array(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879',
       'Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813',
       'McMillan and others, 2013, GRL, doi:10.1002/grl.50689',
       'Kim and others, 2016, TC, doi:10.5194/tc-10-2971-2016',
       'Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085',
       'Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557',
       'Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063',
       'Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36',
       'Smith and others, 2017, TC, doi:10.5194/tc-11-451-2017'],
      dtype=object)

In [319]:
# Replace TC with Cryosphere
lake_locations = lake_locations.replace('Kim and others, 2016, TC, doi:10.5194/tc-10-2971-2016', 'Kim and others, 2016, Cryosphere, doi:10.5194/tc-10-2971-2016')
lake_locations = lake_locations.replace('Smith and others, 2017, TC, doi:10.5194/tc-11-451-2017', 'Smith and others, 2017, Cryosphere, doi:10.5194/tc-11-451-2017')
# Replace GRL with Geophys. Res. Lett.
lake_locations = lake_locations.replace('McMillan and others, 2013, GRL, doi:10.1002/grl.50689', 'McMillan and others, 2013, Geophys. Res. Lett., doi:10.1002/grl.50689')
# Ensure replacements worked as expected
lake_locations['cite'].unique()

array(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879',
       'Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813',
       'McMillan and others, 2013, Geophys. Res. Lett., doi:10.1002/grl.50689',
       'Kim and others, 2016, Cryosphere, doi:10.5194/tc-10-2971-2016',
       'Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085',
       'Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557',
       'Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063',
       'Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36',
       'Smith and others, 2017, Cryosphere, doi:10.5194/tc-11-451-2017'],
      dtype=object)

In [320]:
# Find area of previously identified lakes inventoried in SF18
# to use as guesstimate for lakes without this information or figures 
# allowing for closer guesstimate
np.mean(SF18_outlines['area (m^2)'])/1e6

202.21730126890483

In [338]:
# Add entries for newer lakes from publications not included in Siegfried & Fricker (2018) inventory 
# (using approximated centroid point when outline is unavailable)

# Smith 2009 Recovery_8 (dropped from Siegfried & Fricker (2018) outlines) to see if there's activity
lake_gdf = S09_outlines[S09_outlines['Name'] == 'Recovery_8']
name = lake_gdf['Name'].values[0]
# S09 outline inventory uses 3D polygons with z dimension vs. 2D polygons in SF18 inventory
# Extract the point values that define the perimeter of the polygon to make polygon without third z dimension
# Extract 2D coordinates (X, Y) from the 3D polygon (X, Y, Z)
xy_coords = [(x, y) for x, y, z in lake_gdf.geometry.values[0].exterior.coords]
# Create a new 2D polygon from these coordinates
lake_poly_2d = Polygon(xy_coords)
geometry = lake_poly_2d
area = lake_gdf['area (m^2)'].values[0]
# Store citation info from another lake from the same S09 study in the SF18 citation format
cite = SF18_outlines[SF18_outlines['name'] == 'Bindschadler_1'].cite.values[0]
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, 'False']], columns=lake_locations.columns)
# Convert the DataFrame to a GeoDataFrame
# gdf = gpd.GeoDataFrame(df, crs='EPSG:3031')
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# gdf_diff = gdf[~gdf['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

In [306]:
# Livingstone and others, 2022 active lakes points (no outlines available):
# Wingham and others, 2006 (within Livingstone and others, 2022 inventory)
# Lakes L1, U1, U2, U3
areas = [600e6, 200e6, 200e6, 400e6]  # Lake L1 area estimated in paper, but U1-3 were guestimated from Fig. 1
for i in range(0, 4):
    name = L22_activelake_points_new.iloc[i]['Name']
    lon = L22_activelake_points_new.iloc[i]['Lon. oE']
    lat = L22_activelake_points_new.iloc[i]['Lat.  oN']
    geometry = Point(ll2ps(lon, lat)).buffer(math.sqrt(areas[i] / math.pi))
    area = areas[i]
    cite = 'Wingham and others, 2006, Nature, doi:10.1038/nature04660'
    CS2_SARIn = 'False'
    # Make entry into pandas dataframe to concatenate to inventory
    df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
    # Ensure that new entry isn't already in inventory before adding
    df_diff = df[~df['name'].isin(lake_locations['name'])]
    # Add entry to inventory
    lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)
# Delete list
del areas

# N. Young personal comm. (within Wright & Siegert, 2012 and Livingstone and others, 2022 inventories)
# Lakes Site A, Site B, Site C
area = 200e6
for i in range(6, 9):
    name = L22_activelake_points_new.iloc[i]['Name']
    lon = L22_activelake_points_new.iloc[i]['Lon. oE']
    lat = L22_activelake_points_new.iloc[i]['Lat.  oN']
    geometry = Point(ll2ps(lon, lat)).buffer(math.sqrt(area / math.pi))
    cite = 'Wright & Siegert, 2012, Antarct. Sci., doi:10.1017/S095410201200048X'
    CS2_SARIn = 'True'
    # Make entry into pandas dataframe to concatenate to inventory
    df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
    # Ensure that new entry isn't already in inventory before adding
    df_diff = df[~df['name'].isin(lake_locations['name'])]
    # Add entry to inventory
    lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# Hoffman and others, 2020 (within Livingstone and others, 2022 inventory)
# Western Thwaites (WT) lake
area = SF18_outlines[SF18_outlines['name'] == 'Thw_142']['area (m^2)'].values[0]  # Using nearby Thwaites lake of similar area from Fig. 1
for i in range(9, 10):
    name = L22_activelake_points_new.iloc[i]['Name']
    lon = L22_activelake_points_new.iloc[i]['Lon. oE']
    lat = L22_activelake_points_new.iloc[i]['Lat.  oN']
    geometry = Point(ll2ps(lon, lat)).buffer(math.sqrt(area / math.pi))
    cite = 'Hoffman and others, 2020, Cryosphere, doi:10.5194/tc-14-4603-2020'
    CS2_SARIn = 'True'
    # Make entry into pandas dataframe to concatenate to inventory
    df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
    # Ensure that new entry isn't already in inventory before adding
    df_diff = df[~df['name'].isin(lake_locations['name'])]
    # Add entry to inventory
    lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# Livingstone and others, 2022 inventory contains a point for Haynes Glacier system of four active lakes detailed in 
# Hoffman and others, 2020; however, only one point for four lakes and point is adjacent to four lakes, not one of four lakes, so adding manually
# Haynes Glacier (HG) lakes point location and area estimated from Hoffman and others, 2020 supplement Fig. S1 (https://doi.org/10.5194/tc-14-4603-2020-supplement)
# TL96
name = 'TL96'
area = 25e6
geometry = Point(-1439000, -544000).buffer(math.sqrt(area / math.pi))
cite = 'Hoffman and others, 2020, Cryosphere, doi:10.5194/tc-14-4603-2020'
CS2_SARIn = 'True'
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# TL108
name = 'TL108'
area = 15e6
geometry = Point(-1427500, -542000).buffer(math.sqrt(area / math.pi))
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# TL115
name = 'TL115'
area = 5e6
geometry = Point(-1422500, -537500).buffer(math.sqrt(area / math.pi))
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# TL122
name = 'TL122'
area = 10e6
geometry = Point(-1419000, -532500).buffer(math.sqrt(area / math.pi))
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# Siegfried and Fricker, 2021 (publication not included in Livingstone and others, 2022 inventory)
# Lower Subglacial Lake Mercer (LSLM)
name = 'LowerMercerSubglacialLake'
area = 10e6
geometry = Point(-308000, -509000).buffer(math.sqrt(area / math.pi))
cite = 'Siegfried and Fricker, 2021, Geophys. Res. Lett., doi:10.1029/2020GL091089'
CS2_SARIn = 'True'
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# Lower Subglacial Lake Conway (LSLC)
name = 'LowerSubglacialLakeConway'
area = 10e6
geometry = Point(-282000, -502000).buffer(math.sqrt(area / math.pi))
cite = 'Siegfried and Fricker, 2021, Geophys. Res. Lett., doi:10.1029/2020GL091089'
CS2_SARIn = 'True'
# Make entry into pandas dataframe to concatenate to inventory
df = pd.DataFrame([[name, geometry, area, cite, CS2_SARIn]], columns=lake_locations.columns)
# Ensure that new entry isn't already in inventory before adding
df_diff = df[~df['name'].isin(lake_locations['name'])]
# Add entry to inventory
lake_locations = pd.concat([lake_locations, df_diff], ignore_index=True)

# Drop outlines from lake_locations that are duplicative of SiegfiedFricker2018_outlines
lake_locations_notSF18 = lake_locations.copy(deep=True)
lake_locations_notSF18.drop(lake_locations_notSF18.iloc[0:131].index, inplace=True)

# Sort geopandas geodataframe to alphabetize by lake name and reset index
# lake_locations_notSF18 = lake_locations_notSF18.sort_values(by=['name'], ignore_index=True)

# Reset the index, dropping the old one
lake_locations_notSF18 = lake_locations_notSF18.reset_index(drop=True)

In [282]:
# Display geodataframe
lake_locations

,name,geometry,area (m^2),cite,CS2_SARIn
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
...,...,...,...,...,...
141,TL108,"POLYGON ((-1425314.903 -542000.000, -1425325.4...",1.500000e+07,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
142,TL115,"POLYGON ((-1421238.434 -537500.000, -1421244.5...",5.000000e+06,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
143,TL122,"POLYGON ((-1417215.876 -532500.000, -1417224.4...",1.000000e+07,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
144,LowerMercerSubglacialLake,"POLYGON ((-306215.876 -509000.000, -306224.467...",1.000000e+07,"Siegfried and Fricker, 2021, Geophys. Res. Let...",True


In [135]:
# Display geodataframe
lake_locations_notSF18

,name,geometry,area (m^2),cite,CS2_SARIn
0,Recovery_8,"POLYGON ((-74016.771 896653.746, -73507.172 89...",NaN,"Smith and others, 2009, J. Glac., doi:10.3189/...",False
1,L1,POINT (1273770.727 -1199082.221),NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False
2,U1,POINT (1085448.426 -1085448.426),NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False
3,U2,POINT (1040017.716 -1068353.903),NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False
4,U3,POINT (1010990.830 -1044355.237),NaN,"Wingham and others, 2006, Nature, doi:10.1038/...",False
5,“Site A”,POINT (2255000.417 -935896.682),NaN,"Wright & Siegert, 2012, Antarct. Sci., doi:10....",True
6,“Site B”,POINT (2231263.202 -971570.334),NaN,"Wright & Siegert, 2012, Antarct. Sci., doi:10....",True
7,“Site C”,POINT (2225613.155 -992769.171),NaN,"Wright & Siegert, 2012, Antarct. Sci., doi:10....",True
8,WT,POINT (-1265566.674 -563466.587),NaN,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True
9,TL96,POINT (-1439000.000 -544000.000),NaN,"Hoffman and others, 2020, Cryosphere, doi:10.5...",True


In [ ]:
# Code to to explore and reconcile outline inventories

In [39]:
S09_outlines

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08
...,...,...,...,...
119,Whillans_6,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-451544.869 -488823.261 1.000, -45...",7.458477e+07
120,Whillans_7,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-543163.376 -500759.165 1.000, -54...",7.696570e+07
121,Whillans_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-654478.748 -281124.560 1.000, -65...",1.625714e+08
122,Wilkes_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((2214185.180 -666018.604 1.000, 221...",5.880773e+08


In [41]:
for idx in range(len(SF18_outlines['cite'].unique())): 
    length = len(SF18_outlines[SF18_outlines['cite'].str.contains(SF18_outlines['cite'].unique()[idx])])
    print(SF18_outlines['cite'].unique()[idx])
    print(length)

Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879
97
Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813
7
McMillan and others, 2013, GRL, doi:10.1002/grl.50689
1
Kim and others, 2016, TC, doi:10.5194/tc-10-2971-2016
3
Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085
1
Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557
8
Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063
9
Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36
1
Smith and others, 2017, TC, doi:10.5194/tc-11-451-2017
4


In [42]:
SF18_outlines[SF18_outlines['cite'] == 'Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813']

,name,geometry,area (m^2),cite
23,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
41,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
80,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
81,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
93,MercerSubglacialLake,"POLYGON ((-299950.018 -500675.034, -299949.971...",1.432030e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
123,UpperSubglacialLakeConway,"POLYGON ((-370200.008 -533925.036, -370200.008...",1.866440e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
125,WhillansSubglacialLake,"POLYGON ((-284324.993 -560675.019, -284200.010...",6.196906e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."


In [43]:
SF18_outlines[SF18_outlines['cite'] == 'McMillan and others, 2013, GRL, doi:10.1002/grl.50689']

,name,geometry,area (m^2),cite
25,Cook_E2,"POLYGON ((765592.392 -1714713.856, 765682.174 ...",2.680933e+08,"McMillan and others, 2013, GRL, doi:10.1002/gr..."


In [44]:
SF18_outlines[SF18_outlines['cite'] == 'Kim and others, 2016, TC, doi:10.5194/tc-10-2971-2016']

,name,geometry,area (m^2),cite
65,KT1,"POLYGON ((-556189.687 -681400.000, -556000.000...",4.549884e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."


In [45]:
SF18_outlines[SF18_outlines['cite'] == 'Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085']

,name,geometry,area (m^2),cite
82,Lake78,"MULTIPOLYGON (((-257325.009 -525800.047, -2571...",233.002708,"Carter and others, 2013, J. Glac., doi:10.3189..."


In [46]:
SF18_outlines[SF18_outlines['cite'] == 'Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557']

,name,geometry,area (m^2),cite
85,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."
86,Mac2,"POLYGON ((-657250.020 -890550.040, -657249.980...",1.457161e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."
87,Mac3,"POLYGON ((-642325.010 -883925.000, -642449.980...",1.502500e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."
88,Mac4,"POLYGON ((-739324.970 -857799.960, -739199.980...",7.520909e+07,"Fricker and others, 2010, J. Glac., doi:10.318..."
89,Mac5,"POLYGON ((-734575.010 -842800.020, -734449.990...",5.998647e+07,"Fricker and others, 2010, J. Glac., doi:10.318..."
90,Mac6,"POLYGON ((-907727.770 -830662.200, -905516.600...",1.054051e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."
91,Mac7,"POLYGON ((-792290.820 -691503.980, -793860.830...",1.943272e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."
92,Mac8,"POLYGON ((-842816.190 -708487.890, -842383.070...",1.072318e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."


In [47]:
SF18_outlines[SF18_outlines['cite'] == 'Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063']

,name,geometry,area (m^2),cite
101,Rec1,"POLYGON ((-448074.984 859324.972, -447949.975 ...",1.046839e+09,"Fricker and others, 2014, J. Glac., doi:10.318..."
102,Rec10,"POLYGON ((140924.994 903449.944, 140924.994 90...",2.931863e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."
103,Rec2,"POLYGON ((-389950.015 915200.035, -389824.971 ...",7.096422e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."
104,Rec3,"POLYGON ((-323075.001 894450.022, -323075.005 ...",6.251871e+07,"Fricker and others, 2014, J. Glac., doi:10.318..."
105,Rec4,"POLYGON ((-174200.004 944325.029, -174200.000 ...",2.299425e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."
106,Rec5,"POLYGON ((-128199.995 941949.982, -128200.002 ...",2.849787e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."
107,Rec6,"POLYGON ((-90075.003 920949.990, -90075.003 92...",4.107123e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."
108,Rec8,"POLYGON ((37675.000 782450.023, 37799.998 7824...",2.370994e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."
109,Rec9,"POLYGON ((75300.001 714325.030, 75424.999 7143...",2.358326e+08,"Fricker and others, 2014, J. Glac., doi:10.318..."


In [48]:
SF18_outlines[SF18_outlines['cite'] == 'Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36']

,name,geometry,area (m^2),cite
112,Slessor_23,"POLYGON ((-408685.484 1046792.659, -408498.695...",2.702501e+08,"Siegfried & Fricker, 2018, Ann. Glac., doi:10...."


In [ ]:
# Siegfried and Fricker (2018) only contains 97 Smith and others (2009) citations due to: 
# 124 lakes in Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879
# -7 Fricker & Scambos, 2009, J. Glac., doi:10.3189/002214309788608813 redelineates Mercer/Whillans lakes
# -8 Fricker and others, 2010, J. Glac., doi:10.3189/002214310791968557 redelineates MacAyeal Ice Stream lakes
# -1 Carter and others, 2013, J. Glac., doi:10.3189/2013JoG13J085 redelineates Mercer_1 as Lake78
# -9 Fricker and others, 2014, J. Glac., doi:10.3189/2014JoG14J063 redelineates Recovery Glacier lakes
# -2 Siegfried & Fricker, 2018, Ann. Glac., doi:10.1017/aog.2017.36 redelineates Slessor2 and Slessor3 as Slessor23
# 97 citations to Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879 in Siegfried and Fricker, 2018 inventory

In [52]:
for idx in range(len(L22_activelake_points['References'].unique())): 
    length = len(L22_activelake_points[L22_activelake_points['References'].str.contains(L22_activelake_points['References'].unique()[idx], regex=False)])
    print(L22_activelake_points['References'].unique()[idx])
    print(length)

Gray et al. (2005); Smith et al. (2009)
2
Wingham et al. (2006); Surveyed by ICECAP in 2008/09 (Wright et al., 2012)
1
Wingham et al. (2006)
4
Fricker et al. (2007), Blankenship et al. (2009), Siegfried et al., (2014), Siegfried & Fricker (2018)
1
Fricker et al. (2007); Siegfried et al. (2014); Siegfried & Fricker (2018)
1
Lake Engelhardt (Fricker et al., 2007); Carter et al., (2013); Siegfried & Fricker (2018)
1
Fricker et al. (2007); Siegfried & Fricker (2018)
2
Lake Whillans, (Fricker et al., 2007); Geophysical observations made at low stand (Christianson et al., 2012 and Horgan et al. 2012); Carter et al., (2013); Siegfried et al. (2014); Siegfried & Fricker (2018).
1
Lake Conway (Fricker et al., 2007); Carter et al., (2013); Siegfried et al. (2014); Siegfried & Fricker (2018)
1
Stearns et al., (2008)
2
Smith et al. (2009)
114
Smith et al. (2009); (Welch et al., 2009)
1
Smith et al. (2009); McMillan et al., (2013); Flament et al. (2014); Li et al., (2020)
1
Smith et al. (2009); Sie

In [53]:
L22_activelake_points.loc[L22_activelake_points['References'].str.contains('Smith et al. (2009)', regex=False)]

,Name,Lat. oN,Lon. oE,Lake Type,References,Prior Inventory
0,Bindschadler5,-80.610,-123.994,Active,Gray et al. (2005); Smith et al. (2009),Wright & Siegert (2012)
1,Kamb10,-81.446,-120.134,Active,Gray et al. (2005); Smith et al. (2009),Wright & Siegert (2012)
15,Academy1 (Foundation 1),-84.135,-61.337,Active,Smith et al. (2009),Smith et al. (2009) / Wright & Siegert (2012)
16,Academy2 (Foundation 2),-84.536,-57.445,Active,Smith et al. (2009),Smith et al. (2009) / Wright & Siegert (2012)
17,Academy3 (Foundation3),-84.608,-59.082,Active,Smith et al. (2009),Smith et al. (2009) / Wright & Siegert (2012)
...,...,...,...,...,...,...
122,Whillans8,-83.504,-113.601,Active,Smith et al. (2009),Smith et al. (2009) / Wright & Siegert (2012)
123,Wilkes1,-68.812,106.715,Active,Smith et al. (2009); Siegfried & Fricker (2018),Smith et al. (2009) / Wright & Siegert (2012)
124,Wilkes2,-68.703,121.566,Active,Smith et al. (2009),Smith et al. (2009) / Wright & Siegert (2012)
126,Mac4,-79.736,-139.000,Active,"Fricker et al. (2010), part of MacAyeal3 in Sm...",Wright & Siegert (2012)


In [54]:
SF18_outlines['name'].tolist()

['Bindschadler_1',
 'Bindschadler_2',
 'Bindschadler_3',
 'Bindschadler_4',
 'Bindschadler_5',
 'Bindschadler_6',
 'Byrd_1',
 'Byrd_2',
 'Byrd_s1',
 'Byrd_s10',
 'Byrd_s11',
 'Byrd_s12',
 'Byrd_s13',
 'Byrd_s14',
 'Byrd_s15',
 'Byrd_s2',
 'Byrd_s3',
 'Byrd_s4',
 'Byrd_s5',
 'Byrd_s6',
 'Byrd_s7',
 'Byrd_s8',
 'Byrd_s9',
 'ConwaySubglacialLake',
 'Cook_E1',
 'Cook_E2',
 'David_1',
 'David_s1',
 'David_s2',
 'David_s3',
 'David_s4',
 'David_s5',
 'EAP_1',
 'EAP_2',
 'EAP_3',
 'EAP_4',
 'EAP_5',
 'EAP_6',
 'EAP_7',
 'EAP_8',
 'EAP_9',
 'EngelhardtSubglacialLake',
 'Foundation_1',
 'Foundation_10',
 'Foundation_11',
 'Foundation_12',
 'Foundation_13',
 'Foundation_14',
 'Foundation_15',
 'Foundation_16',
 'Foundation_2',
 'Foundation_3',
 'Foundation_4',
 'Foundation_5',
 'Foundation_6',
 'Foundation_7',
 'Foundation_8',
 'Foundation_9',
 'Foundation_N1',
 'Foundation_N2',
 'Foundation_N3',
 'Institute_E1',
 'Institute_E2',
 'Institute_W1',
 'Institute_W2',
 'KT1',
 'KT2',
 'KT3',
 'Kamb_1

In [55]:
# View head of Smith and others, 2009 static outline geodataframe
S09_outlines.head()

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08


In [56]:
S09_outlines[S09_outlines.Name == 'Recovery_8']

,Name,Description,geometry,area (m^2)
98,Recovery_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-74016.771 896653.746 1.000, -7350...",4.355089e+08


In [65]:
# Find outlines in SiegfriedFricker2018_outlines static outline geodataframe with Smith and others, 2009 citation
# These lakes have not been redefined with a new outline and can be analysed as is
SF18_outlines_S09only = SF18_outlines[SF18_outlines['cite'].isin(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'])]
SF18_outlines_S09only

,name,geometry,area (m^2),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [66]:
# Find outlines in SiegfriedFricker2018_outlines geodataframe with citations not matching Smith and others, 2009
# These are eitherly newly identified lakes or lakes that have been redefined with new static outlines 
SF18_outlines_SF18only = SF18_outlines[~SF18_outlines['cite'].isin(['Smith and others, 2009, J. Glac., doi:10.3189/002214309789470879'])]
SF18_outlines_SF18only

,name,geometry,area (m^2),cite
23,ConwaySubglacialLake,"POLYGON ((-312825.002 -511425.001, -312699.997...",2.669973e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
25,Cook_E2,"POLYGON ((765592.392 -1714713.856, 765682.174 ...",2.680933e+08,"McMillan and others, 2013, GRL, doi:10.1002/gr..."
41,EngelhardtSubglacialLake,"POLYGON ((-271824.984 -628674.969, -271699.995...",3.577632e+08,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
65,KT1,"POLYGON ((-556189.687 -681400.000, -556000.000...",4.549884e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
80,Lake10,"POLYGON ((-227949.981 -569549.997, -228075.000...",2.587121e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
81,Lake12,"POLYGON ((-224449.970 -604674.969, -224449.970...",6.689842e+07,"Fricker & Scambos, 2009, J. Glac., doi:10.3189..."
82,Lake78,"MULTIPOLYGON (((-257325.009 -525800.047, -2571...",2.330027e+02,"Carter and others, 2013, J. Glac., doi:10.3189..."
85,Mac1,"POLYGON ((-629350.010 -889125.010, -629325.010...",1.563754e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."


In [67]:
len(SF18_outlines) == len(SF18_outlines_S09only) + len(SF18_outlines_SF18only)

True

In [73]:
# Find spatial intersections between two static outline geodataframes to indicate groups of outlines that are redefined outlines of one lake
S09_SF18_intersections = find_intersections(S09_outlines, SF18_outlines)

In [74]:
# Extract intersecting indicies from each static outline geodataframe for later use
S09_intersection_indices = [inner_list[0] for inner_list in S09_SF18_intersections]
SF18_intersection_indices = [inner_list[1] for inner_list in S09_SF18_intersections]

In [93]:
# Store the lake names from Smith and others, 2009 and Siegfried & Fricker, 2018 that intersect
# Store 'name' column for rows in gdf1 corresponding to indices in indices1
S09_intersections_names = S09_outlines.iloc[S09_intersection_indices]['Name']

# Store 'name' column for rows in gdf2 corresponding to indices in indices2
SF18_intersections_names = SF18_outlines.iloc[SF18_intersection_indices]['name']

# Concatenate into dataframe
intersecting_lake_names = pd.concat([S09_intersections_names.reset_index(drop=True), SF18_intersections_names.reset_index(drop=True)], axis=1)
intersecting_lake_names.columns = ['S09', 'SF18']

# Filtering rows where column1 and column2 differ indicating lake was redelineated
intersecting_lake_names_diff = intersecting_lake_names[intersecting_lake_names['S09'] != intersecting_lake_names['SF18']]

In [121]:
print(intersecting_lake_names.to_string())

                S09                       SF18
0    Bindschadler_1             Bindschadler_1
1    Bindschadler_1                       Mac7
2    Bindschadler_2             Bindschadler_2
3    Bindschadler_2                       Mac8
4    Bindschadler_3             Bindschadler_3
5    Bindschadler_4             Bindschadler_4
6    Bindschadler_5             Bindschadler_5
7    Bindschadler_6             Bindschadler_6
8            Byrd_1                     Byrd_1
9            Byrd_2                     Byrd_2
10         Byrd_s10                   Byrd_s10
11         Byrd_s11                   Byrd_s11
12         Byrd_s12                   Byrd_s12
13         Byrd_s13                   Byrd_s13
14         Byrd_s14                   Byrd_s14
15         Byrd_s15                   Byrd_s15
16          Byrd_s1                    Byrd_s1
17          Byrd_s2                    Byrd_s2
18          Byrd_s3                    Byrd_s3
19          Byrd_s4                    Byrd_s4
20          B

In [93]:
print(intersecting_lake_names_diff)

                S09                       SF18
1    Bindschadler_1                       Mac7
3    Bindschadler_2                       Mac8
59     InstituteE_1               Institute_E1
60     InstituteE_2               Institute_E2
61     InstituteW_1               Institute_W1
62     InstituteW_2               Institute_W2
63      KambTrunk_1                        KT1
78       Macayeal_1                       Mac1
79       Macayeal_2                       Mac2
80       Macayeal_3                       Mac4
81       Macayeal_3                       Mac5
82       Macayeal_4                       Mac6
83         Mercer_1                     Lake78
84         Mercer_2       MercerSubglacialLake
92      Recovery_10                       Rec9
93      Recovery_11                      Rec10
94       Recovery_1                       Rec1
95       Recovery_2                       Rec1
96       Recovery_3                       Rec2
97       Recovery_4                       Rec3
98       Reco

In [94]:
# Extract rows with intersection between the Smith and others, 2009 static outlines and the static outlines in Siegfried & Fricker, 2018
# These are lakes that were defined in Smith and others, 2009 and subsequently renamed or redelineated by publications after Smith and others, 2009
# We will analyze as a unified polygon of all past static outlines for each lake
S09_intersections = S09_outlines[S09_outlines.index.isin(S09_intersection_indices)] 
S09_intersections

,Name,Description,geometry,area (m^2)
0,Bindschadler_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-792264.327 -691480.857 1.000, -79...",1.943146e+08
1,Bindschadler_2,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-842788.063 -708464.240 1.000, -84...",1.072249e+08
2,Bindschadler_3,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-874893.221 -654533.044 1.000, -87...",1.404559e+08
3,Bindschadler_4,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-828821.778 -584874.415 1.000, -82...",2.816411e+08
4,Bindschadler_5,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-858067.460 -573467.564 1.000, -85...",3.923966e+08
...,...,...,...,...
119,Whillans_6,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-451544.869 -488823.261 1.000, -45...",7.458477e+07
120,Whillans_7,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-543163.376 -500759.165 1.000, -54...",7.696570e+07
121,Whillans_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-654478.748 -281124.560 1.000, -65...",1.625714e+08
122,Wilkes_1,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((2214185.180 -666018.604 1.000, 221...",5.880773e+08


In [95]:
# Do the same for the Siegfried & Fricker, 2018 static outlines
SF18_intersections = SF18_outlines[SF18_outlines.index.isin(SF18_intersection_indices)] 
SF18_intersections

,name,geometry,area (m^2),cite
0,Bindschadler_1,"POLYGON ((-792264.327 -691480.857, -791281.458...",1.943146e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
1,Bindschadler_2,"POLYGON ((-842788.063 -708464.240, -842354.948...",1.072249e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
2,Bindschadler_3,"POLYGON ((-874893.221 -654533.044, -876415.673...",1.404559e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
3,Bindschadler_4,"POLYGON ((-828821.778 -584874.415, -828822.032...",2.816411e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
4,Bindschadler_5,"POLYGON ((-858067.460 -573467.564, -858714.391...",3.923966e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
...,...,...,...,...
126,Whillans_6,"POLYGON ((-451544.869 -488823.261, -451209.964...",7.458477e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
127,Whillans_7,"POLYGON ((-543163.376 -500759.165, -542800.367...",7.696570e+07,"Smith and others, 2009, J. Glac., doi:10.3189/..."
128,Whillans_8,"POLYGON ((-654478.748 -281124.560, -653777.327...",1.625714e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."
129,Wilkes_1,"POLYGON ((2214185.180 -666018.604, 2214317.389...",5.880773e+08,"Smith and others, 2009, J. Glac., doi:10.3189/..."


In [96]:
# Extract rows with no intersection between the Smith and others, 2009 and the renamed or redefined static outlines in Siegfried & Fricker, 2018
# These are new lakes that were delineated by Smith and others, 2009 and not subsequently rename or redelineated
# These lakes only have one static outline and can be analysed as is
S09_non_intersections = S09_outlines[~S09_outlines.index.isin(S09_intersection_indices)] 
S09_non_intersections

,Name,Description,geometry,area (m^2)
98,Recovery_8,<table border=1 width='100%' > <tr><td><bf>Cam...,"POLYGON Z ((-74016.771 896653.746 1.000, -7350...",4.355089e+08


In [97]:
# Extract rows with no intersection between the Smith and others, 2009 and the renamed or redefined static outlines in Siegfried & Fricker, 2018
# These are new lakes that were added by publications subsequent to Smith and others, 2009
# These lakes only have one static outline and can be analysed as is
SF18_non_intersections = SF18_outlines[~SF18_outlines.index.isin(SF18_intersection_indices)] 
SF18_non_intersections

,name,geometry,area (m^2),cite
66,KT2,"POLYGON ((-441794.803 -712600.000, -441600.000...",3.315274e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
67,KT3,"POLYGON ((-399646.273 -716600.000, -399600.000...",4.055444e+07,"Kim and others, 2016, TC, doi:10.5194/tc-10-29..."
87,Mac3,"POLYGON ((-642325.010 -883925.000, -642449.980...",1.502500e+08,"Fricker and others, 2010, J. Glac., doi:10.318..."
117,Thw_124,"POLYGON ((-1419402.000 -431819.000, -1421479.0...",5.761346e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."
118,Thw_142,"POLYGON ((-1404705.000 -410106.000, -1404692.0...",1.607689e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."
119,Thw_170,"POLYGON ((-1380997.000 -398714.000, -1382313.0...",1.940960e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."
120,Thw_70,"POLYGON ((-1476557.000 -449098.000, -1474606.0...",3.540275e+08,"Smith and others, 2017, TC, doi:10.5194/tc-11-..."


In [106]:
len(S09_outlines) == len(S09_intersections) + len(S09_non_intersections)

True

In [122]:
len(S09_intersections)

123

In [111]:
len(S09_non_intersections)  # This is Recovery_8 that was dropped in SF18 inventory

1

In [107]:
len(SF18_outlines) == len(SF18_intersections) + len(SF18_non_intersections)

True

In [112]:
len(SF18_intersections)  # Lakes from S09 inventory that were either keep as is or redelinated; there is one more than S09_intersections because 

124

In [113]:
len(SF18_non_intersections)  # These are new lakes added in the SF18 inventory from other publications since S09 inventory

7

In [123]:
# Create list of S09 lakes redelineated in SF18
S09_SF18_redelineated_lake_names = [   
    ['Cook_E2', 'Cook_E2'],
    ['KambTrunk_1', 'KT1'],
    ['Macayeal_1', 'Mac1'],
    ['Macayeal_2', 'Mac2'],
    ['Mercer_1', 'Lake78'],
    ['Mercer_2', 'MercerSubglacialLake'],
    ['Recovery_3', 'Rec2'],
    ['Recovery_4', 'Rec3'],
    ['Recovery_5', 'Rec4'], 
    ['Recovery_6', 'Rec5'],
    ['Recovery_7', 'Rec6'],
    ['Recovery_9', 'Rec8'],
    ['Recovery_10', 'Rec9'],
    ['Recovery_11', 'Rec10'],     
    ['Whillans_1', 'EngelhardtSubglacialLake'],            
    ['Whillans_2a', 'Lake12'],
    ['Whillans_2b', 'Lake10'],
    ['Whillans_3', 'WhillansSubglacialLake'],
    ['Whillans_4', 'ConwaySubglacialLake'],
    ['Whillans_5', 'UpperSubglacialLakeConway'],
    # Two S09 lakes converted to one SF18 lake
    [['Slessor_2', 'Slessor_3'], 'Slessor_23'],
    [['Recovery_1', 'Recovery_2'], 'Rec1'],
    # One S09 lake converted to two SF18 lakes
    ['Macayeal_3', ['Mac4', 'Mac5']]
]

In [ ]:
# Add rows for lake candidates I've observed in ICESat-2 data (using approx centroid point until outline is established)
# New observation at ~–450 km, –540 km (polar stereographic x, y)
# Sauthoff2024_outlines.loc[len(Sauthoff2024_outlines)] = ['lower_Whillans6', 'POINT (-450000, -540000)', 'nan', 'nan', 'Sauthoff and others, in prep', 'True'] 

# display dataframe preview
# Sauthoff2024_outlines